# Establishing database connection, running the main query

Grouping fires by cause, creating broader subcategories

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/FPA_FOD_20170508.sqlite')

query_1 = """
        SELECT
            FIRE_CODE, FIRE_NAME, FIRE_YEAR, datetime(DISCOVERY_DATE) AS DISCOVERY_DATE,
            DISCOVERY_DOY, DISCOVERY_TIME, STAT_CAUSE_DESCR, CASE
             WHEN STAT_CAUSE_DESCR = 'Lightning' THEN 'Natural'
             WHEN STAT_CAUSE_DESCR IN ('Campfire', 'Children', 'Debris Burning', 'Equipment Use', 'Fireworks', 'Powerline', 'Railroad', 'Smoking', 'Structure') 
              THEN 'Accidental Human Activity'
             WHEN STAT_CAUSE_DESCR = 'Arson' THEN 'Intentional Human Activity'
             ELSE 'Miscellaneous/Unknown'
            END AS CAUSE_CATEGORY,
            datetime(CONT_DATE) AS CONT_DATE,
            CONT_DOY, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE, STATE, COUNTY
        FROM
            Fires
        WHERE
            COUNTY IS NOT NULL
            AND CONT_DATE IS NOT NULL
        """

df_1 = pd.read_sql_query(query_1, conn)

df_1.head()

,FIRE_CODE,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_DESCR,CAUSE_CATEGORY,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,BJ8K,FOUNTAIN,2005,2005-02-02 00:00:00,33,1300,Miscellaneous,Miscellaneous/Unknown,2005-02-02 00:00:00,33,1730,0.10,A,40.036944,-121.005833,CA,63
1,AAC0,PIGEON,2004,2004-05-12 00:00:00,133,0845,Lightning,Natural,2004-05-12 00:00:00,133,1530,0.25,A,38.933056,-120.404444,CA,61
2,A32W,SLACK,2004,2004-05-31 00:00:00,152,1921,Debris Burning,Accidental Human Activity,2004-05-31 00:00:00,152,2024,0.10,A,38.984167,-120.735556,CA,17
3,None,DEER,2004,2004-06-28 00:00:00,180,1600,Lightning,Natural,2004-07-03 00:00:00,185,1400,0.10,A,38.559167,-119.913333,CA,3
4,None,STEVENOT,2004,2004-06-28 00:00:00,180,1600,Lightning,Natural,2004-07-03 00:00:00,185,1200,0.10,A,38.559167,-119.933056,CA,3


# Adding a column with full state names to the dataframe, as we'll need this for the choropleth map

In [3]:
state_code_to_name = {
    'CA': 'California', 'TX': 'Texas', 'NY': 'New York', 'FL': 'Florida', 'WA': 'Washington',
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky',
    'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan',
    'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska',
    'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NC': 'North Carolina',
    'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
    'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee',
    'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WV': 'West Virginia', 'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

df_1['STATE_NAME'] = df_1['STATE'].map(state_code_to_name)

df_1.head()

df_1.count


<bound method DataFrame.count of        FIRE_CODE                FIRE_NAME  FIRE_YEAR       DISCOVERY_DATE  \
0           BJ8K                 FOUNTAIN       2005  2005-02-02 00:00:00   
1           AAC0                   PIGEON       2004  2004-05-12 00:00:00   
2           A32W                    SLACK       2004  2004-05-31 00:00:00   
3           None                     DEER       2004  2004-06-28 00:00:00   
4           None                 STEVENOT       2004  2004-06-28 00:00:00   
...          ...                      ...        ...                  ...   
669649      None  HARRISON ST / DILLON RD       2015  2015-12-11 00:00:00   
669650      None                   MILE 7       2015  2015-10-28 00:00:00   
669651      None                    HOGAN       2015  2015-10-21 00:00:00   
669652      None          FASHION WY  BAN       2015  2015-11-27 00:00:00   
669653      None                 TOMKI #2       2015  2015-10-07 00:00:00   

        DISCOVERY_DOY DISCOVERY_TIME   STA

# Generating JSON file from the resulting dataframe

In [4]:
df_1.to_json("./../../frontend/public/fires.json", orient = "records", indent = 4)

# Closing the connection to the database

In [5]:
conn.close()